In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import pandas as pd
import numpy as np

In [2]:
imdb_reviews=pd.read_csv("C:/Users/gopes/Machine learning projects/Sentiment Analysis/imdb_reviews.csv")
test_reviews=pd.read_csv("C:/Users/gopes/Machine learning projects/Sentiment Analysis/test_reviews.csv")

In [3]:
imdb_reviews.head()

,Reviews,Sentiment
0,<START this film was just brilliant casting lo...,positive
1,<START big hair big boobs bad music and a gian...,negative
2,<START this has to be one of the worst films o...,negative
3,<START the <UNK> <UNK> at storytelling the tra...,positive
4,<START worst mistake of my life br br i picked...,negative


In [4]:
test_reviews.head()

,Reviews,Sentiment
0,<START please give this one a miss br br <UNK>...,negative
1,<START this film requires a lot of patience be...,positive
2,<START many animation buffs consider <UNK> <UN...,positive
3,<START i generally love this type of movie how...,negative
4,<START like some other people wrote i'm a die ...,positive


In [5]:
word_index = pd.read_csv("C:/Users/gopes/Machine learning projects/Sentiment Analysis/word_indexes.csv")

In [6]:
word_index.head(n=10)

,Words,Indexes
0,tsukino,52009
1,nunnery,52010
2,sonja,16819
3,vani,63954
4,woods,1411
5,spiders,16118
6,hanging,2348
7,woody,2292
8,trawling,52011
9,hold's,52012


In [7]:
word_index=dict(zip(word_index.Words,word_index.Indexes))

In [13]:
word_index["<PAD>"]=0
word_index["<START"]=1
word_index["<UNK>"]=2
word_index["<UNUSED>"]=3

In [9]:
def review_encoder(text):
    arr=[word_index[word] for word in text]
    return arr

In [10]:
train_data,train_labels=imdb_reviews['Reviews'],imdb_reviews['Sentiment']
test_data,test_labels=test_reviews['Reviews'],test_reviews['Sentiment']

In [11]:
train_data=train_data.apply(lambda review:review.split())
test_data=test_data.apply(lambda review:review.split())

In [14]:
train_data=train_data.apply(review_encoder)
test_data=test_data.apply(review_encoder)

In [15]:
def e_sentiments(sentiment):
    if sentiment == 'positive':
        return 1 
    else:
        return 0
    
train_labels=train_labels.apply(e_sentiments)
test_labels=test_labels.apply(e_sentiments)

In [16]:
train_data=keras.preprocessing.sequence.pad_sequences(train_data,value=word_index["<PAD>"],padding='post',maxlen=500)
test_data=keras.preprocessing.sequence.pad_sequences(test_data,value=word_index["<PAD>"],padding='post',maxlen=500)

In [17]:
model=keras.Sequential([keras.layers.Embedding(10000,16,input_length=500),
                      keras.layers.GlobalAveragePooling1D(),
                      keras.layers.Dense(16,activation='relu'),
                      keras.layers.Dense(1,activation='sigmoid')])

In [18]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [38]:
history = model.fit(train_data,train_labels,epochs=40,batch_size=512,validation_data=(test_data,test_labels))

Epoch 1/40
49/49 [==============================] - 2s 31ms/step - loss: 0.0308 - accuracy: 0.9941 - val_loss: 0.6375 - val_accuracy: 0.8581
Epoch 2/40
49/49 [==============================] - 1s 30ms/step - loss: 0.0302 - accuracy: 0.9944 - val_loss: 0.6407 - val_accuracy: 0.8580
Epoch 3/40
49/49 [==============================] - 1s 29ms/step - loss: 0.0297 - accuracy: 0.9946 - val_loss: 0.6470 - val_accuracy: 0.8584
Epoch 4/40
49/49 [==============================] - 1s 29ms/step - loss: 0.0294 - accuracy: 0.9945 - val_loss: 0.6440 - val_accuracy: 0.8571
Epoch 5/40
49/49 [==============================] - 1s 30ms/step - loss: 0.0293 - accuracy: 0.9944 - val_loss: 0.6499 - val_accuracy: 0.8574
Epoch 6/40
49/49 [==============================] - 1s 30ms/step - loss: 0.0288 - accuracy: 0.9946 - val_loss: 0.6507 - val_accuracy: 0.8563
Epoch 7/40
49/49 [==============================] - 1s 30ms/step - loss: 0.0282 - accuracy: 0.9946 - val_loss: 0.6602 - val_accuracy: 0.8574
Epoch 8/40
49

In [20]:
loss,accuracy=model.evaluate(test_data,test_labels)

782/782 [==============================] - 1s 868us/step - loss: 0.3205 - accuracy: 0.8812


In [21]:
test_reviews.head()

,Reviews,Sentiment
0,<START please give this one a miss br br <UNK>...,negative
1,<START this film requires a lot of patience be...,positive
2,<START many animation buffs consider <UNK> <UN...,positive
3,<START i generally love this type of movie how...,negative
4,<START like some other people wrote i'm a die ...,positive


In [22]:
index=np.random.randint(1,1000)

In [23]:
review_case=test_reviews.loc[index]
print(review_case)

Reviews      <START i actually found out about rising via t...
Sentiment                                             positive
Name: 155, dtype: object


In [24]:
review_case=test_data[index]
review_case=np.array([review_case])
if(model.predict(review_case) > 0.5).astype("int32"):
    print("The movie was good")
else:
    print("The movie was ridiculously bad")

The movie was good
